# Lab 0.2: Amdahl's Law in Practice

**Chapter 0: The Parallel Mindset**

This lab explores Amdahl's Law through simulation and real measurements.

## Learning Objectives
- Calculate theoretical speedup limits
- Identify sequential bottlenecks in code
- Understand why 1000 cores doesn't mean 1000x speedup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

## Part 1: Amdahl's Law Formula

$$\text{Speedup} = \frac{1}{S + \frac{P}{N}}$$

Where:
- S = sequential fraction (0 to 1)
- P = parallel fraction (P = 1 - S)
- N = number of processors

In [ ]:
def amdahl_speedup(sequential_fraction, num_processors):
    """Calculate theoretical speedup using Amdahl's Law."""
    S = sequential_fraction
    P = 1 - S
    N = num_processors
    return 1 / (S + P / N)

def max_speedup(sequential_fraction):
    """Maximum possible speedup (infinite processors)."""
    return 1 / sequential_fraction if sequential_fraction > 0 else float('inf')

In [ ]:
# Example calculations
print("Amdahl's Law Examples:")
print("=" * 50)

for seq_pct in [1, 5, 10, 25, 50]:
    seq_frac = seq_pct / 100
    print(f"\n{seq_pct}% sequential code:")
    print(f"  Max possible speedup: {max_speedup(seq_frac):.1f}x")
    for n in [4, 16, 64, 256, 1024]:
        speedup = amdahl_speedup(seq_frac, n)
        efficiency = speedup / n * 100
        print(f"  {n:4d} processors: {speedup:6.2f}x speedup ({efficiency:5.1f}% efficiency)")

## Part 2: Visualizing Amdahl's Law

In [ ]:
# Plot speedup vs number of processors for different sequential fractions
processors = np.logspace(0, 10, 100)  # 1 to 10 billion processors
sequential_fractions = [0.01, 0.05, 0.10, 0.25, 0.50]

plt.figure(figsize=(10, 6))

for seq_frac in sequential_fractions:
    speedups = [amdahl_speedup(seq_frac, n) for n in processors]
    max_sp = max_speedup(seq_frac)
    label = f'{seq_frac*100:.0f}% sequential (max {max_sp:.0f}x)'
    plt.semilogx(processors, speedups, label=label, linewidth=2)

plt.xlabel('Number of Processors', fontsize=12)
plt.ylabel('Speedup', fontsize=12)
plt.title("Amdahl's Law: Sequential Code Limits Parallel Speedup", fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(0, 110)
plt.show()

print("\nKey insight: Even with 1 billion processors, 1% sequential code limits you to 100x speedup.")

## Part 3: Finding Sequential Bottlenecks

Let's create a realistic example with both sequential and parallel components.

In [ ]:
def workload_with_bottleneck(data, sequential_setup_time=0.1):
    """
    Simulated workload with:
    - Sequential setup phase (can't be parallelized)
    - Parallel computation phase (can be parallelized)
    """
    # Sequential setup (e.g., loading data, initializing)
    time.sleep(sequential_setup_time)
    
    # Parallel computation
    result = np.sqrt(data) * np.sin(data) + np.exp(-data / 1000)
    
    return result

# Measure baseline
data = np.random.rand(10_000_000)

start = time.time()
result = workload_with_bottleneck(data, sequential_setup_time=0.1)
total_time = time.time() - start

print(f"Total time: {total_time:.3f}s")
print(f"Sequential portion: ~0.1s ({0.1/total_time*100:.1f}%)")
print(f"Parallel portion: ~{total_time-0.1:.3f}s ({(total_time-0.1)/total_time*100:.1f}%)")
print(f"\nMax theoretical speedup: {max_speedup(0.1/total_time):.1f}x")

## Part 4: GPU Context - Why This Matters

In GPU programming, sequential bottlenecks appear in unexpected places:

1. **Kernel launch overhead**: ~5-10 microseconds per launch (CPU side)
2. **Memory transfers**: CPU-GPU data movement is sequential
3. **Python overhead**: GIL, object creation, etc.
4. **Reduction operations**: Final accumulation step

In [ ]:
# Simulate GPU kernel launch overhead impact
def simulate_gpu_workload(compute_time_ms, num_kernels, launch_overhead_us=10):
    """
    Simulate GPU workload with kernel launch overhead.
    
    Args:
        compute_time_ms: Total GPU compute time in milliseconds
        num_kernels: Number of separate kernel launches
        launch_overhead_us: Overhead per launch in microseconds
    """
    compute_time_s = compute_time_ms / 1000
    launch_time_s = (num_kernels * launch_overhead_us) / 1_000_000
    
    total_time = compute_time_s + launch_time_s
    sequential_fraction = launch_time_s / total_time
    
    return {
        'total_time_ms': total_time * 1000,
        'compute_time_ms': compute_time_ms,
        'launch_overhead_ms': launch_time_s * 1000,
        'sequential_fraction': sequential_fraction,
        'max_speedup': max_speedup(sequential_fraction) if sequential_fraction > 0 else float('inf')
    }

print("Impact of kernel launch overhead:")
print("=" * 60)

scenarios = [
    (100, 10, "Large kernels (10 launches, 100ms compute)"),
    (10, 100, "Medium kernels (100 launches, 10ms compute)"),
    (1, 1000, "Small kernels (1000 launches, 1ms compute)"),
    (0.1, 10000, "Tiny kernels (10k launches, 0.1ms compute)"),
]

for compute_ms, num_kernels, desc in scenarios:
    result = simulate_gpu_workload(compute_ms, num_kernels)
    print(f"\n{desc}:")
    print(f"  Launch overhead: {result['launch_overhead_ms']:.2f}ms ({result['sequential_fraction']*100:.1f}% of total)")
    print(f"  Theoretical max speedup from more GPU cores: {result['max_speedup']:.1f}x")

## Exercise: Calculate Your Speedup

You have a training pipeline with:
- Data loading: 200ms (CPU, sequential)
- Preprocessing: 50ms (CPU, sequential)  
- Forward pass: 100ms (GPU, parallel)
- Backward pass: 150ms (GPU, parallel)
- Optimizer step: 30ms (GPU, parallel)

Calculate the maximum speedup from a faster GPU.

In [ ]:
# Your calculation here
data_loading = 200
preprocessing = 50
forward = 100
backward = 150
optimizer = 30

sequential_time = data_loading + preprocessing
parallel_time = forward + backward + optimizer
total_time = sequential_time + parallel_time

sequential_fraction = sequential_time / total_time

print(f"Sequential time: {sequential_time}ms ({sequential_fraction*100:.1f}%)")
print(f"Parallel time: {parallel_time}ms ({(1-sequential_fraction)*100:.1f}%)")
print(f"\nMax speedup from infinitely fast GPU: {max_speedup(sequential_fraction):.2f}x")
print(f"\nTo improve further, you need to optimize: data loading and preprocessing!")

## Key Takeaways

1. **Amdahl's Law sets hard limits**: No amount of parallelism can overcome sequential bottlenecks
2. **Find the sequential parts first**: Profile before optimizing
3. **Small sequential fractions matter at scale**: 1% sequential = max 100x speedup
4. **GPU speedup is bounded by CPU overhead**: Kernel launches, data transfers, Python code